In [3]:
import numpy as np
import pandas as pd
import os
assayfile = 'ACE_CHEMBL1808_IC50_843bioactivity.csv'
df = pd.read_csv(assayfile, delimiter=';')
df = df.replace(to_replace='None', value=np.nan)
print(df.columns)
print(df.shape)

Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties',
       'Action Type'],
      dtype='object')
(8

In [4]:
subdf = df[['Molecule ChEMBL ID', 'Molecular Weight', '#RO5 Violations', \
    'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value', 'Standard Units']]
subdf.describe(include='all')

Molecule ChEMBL ID  Molecular Weight #RO5 Violations  \
count                 843        843.000000             836   
unique                717               NaN               4   
top            CHEMBL1560               NaN               0   
freq                   30               NaN             736   
mean                  NaN        367.858209             NaN   
std                   NaN        119.606807             NaN   
min                   NaN        132.120000             NaN   
25%                   NaN        267.835000             NaN   
50%                   NaN        376.450000             NaN   
75%                   NaN        436.510000             NaN   
max                   NaN       1174.440000             NaN   

                                  Smiles Standard Type Standard Relation  \
count                                843           843               820   
unique                               717             1                 2   
top     C[C@H](CS)C(=O)N1CCC[C@H]1C(=O)O          IC50               '='   
freq                                  30           843               758   
mean                                 NaN           NaN               NaN   
std                                  NaN           NaN               NaN   
min                                  NaN           NaN               NaN   
25%                                  NaN           NaN               NaN   
50%                                  NaN           NaN               NaN   
75%                                  NaN           NaN               NaN   
max                                  NaN           NaN               NaN   

        Standard Value Standard Units  
count     8.200000e+02            837  
unique             NaN              1  
top                NaN             nM  
freq               NaN            837  
mean      2.743672e+06            NaN  
std       4.936111e+07            NaN  
min       2.900000e-02            NaN  
25%       1.201500e+01            NaN  
50%       3.000000e+02            NaN  
75%       1.000000e+04            NaN  
max       1.000000e+09            NaN

## Clean up dataset

- Remove molecules without `Smiles`
- Fix `Standard Units`:
    + Remove `nan`
    + Adjust `µM` to `nM`: *1000
    + Adjust `ug.mL-1` to `nM`: 
        - Convert to `mol` using `Molecular Weight` or `Smiles`
        - Convert to `L`

Remove molecules without SMILES

In [5]:
subdf = df.drop_duplicates(subset=['Molecule ChEMBL ID'])
print(subdf.shape)
sele_df = subdf[~subdf['Smiles'].isna()]
print(sele_df.shape)
sele_df = sele_df[~sele_df['Standard Value'].isna()]
print(sele_df.shape)

(717, 46)
(717, 46)
(695, 46)


Adjust IC50 values

In [6]:
sele_df['Standard Units'].unique()

array(['nM'], dtype=object)

In [7]:
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import Descriptors
def adjust_ug_ml(x):
    if x['Standard Units']=='ug.mL-1':
        mw = Descriptors.MolWt(Chem.MolFromSmiles(x['Smiles']))
        tmp = x['Standard Value'] / mw * 10**6
        return tmp
    else:
        return x['Standard Value']

In [8]:
sele_df = sele_df[~sele_df['Standard Value'].isna()]
sele_df['Standard Value'] = sele_df['Standard Value'].astype('float')
sele_df['Adjusted_IC50'] = sele_df.apply(lambda x: x['Standard Value']*1000 if x['Standard Units']=='µM' \
                                else x['Standard Value'], axis=1)
sele_df['Adjusted_IC50'] = sele_df.apply(lambda x: adjust_ug_ml(x), axis=1)
sele_df = sele_df.drop(['Standard Units'], axis=1)
print(sele_df.shape)

(695, 46)


## Output SMILES and IC50 values

In [9]:
sele_df['Standard Relation'].unique()

array(["'='", "'>'"], dtype=object)

In [10]:
sele_df = sele_df[~sele_df['Standard Relation'].isna()]
print(sele_df.shape)

(695, 46)


In [11]:
eq_df = sele_df[sele_df['Standard Relation'].str.contains('=')]
for idx, row in eq_df.iterrows():
    print(row['Molecule ChEMBL ID'], row['Smiles'], row['Adjusted_IC50'])
    break
print(eq_df.shape)

CHEMBL114658 CC1(C)CCCC(NC(=O)[C@@H](S)Cc2ccccc2)C(=O)N1CC(=O)O 12.0
(635, 46)


In [12]:
with open('cleaned_bioactivities.txt', 'w') as file:
    eq_df = sele_df[sele_df['Standard Relation'].str.contains('=')]
    for idx, row in eq_df.iterrows():
        file.write(row['Molecule ChEMBL ID'] \
            +','+row['Smiles'] \
            +','+f"{row['Adjusted_IC50']:.5f}"+os.linesep)

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 24
plt.rcParams['lines.linewidth'] = 2
custom_params = {"axes.spines.right": True, "axes.spines.top": True}
plt.figure(figsize=(10, 8))
bins = [0, 10, 100, 1e3, 1e4, 1e5, 1e6, 1e7]
normal_ic50 = sele_df[sele_df['Standard Relation'].str.contains('=')]['Adjusted_IC50'].values
sns.displot(normal_ic50, bins=bins, stat='probability', facet_kws=dict(despine=False), )
plt.xticks(bins)
plt.xscale('log')
plt.axis('on')
ax = plt.gca()
ax.tick_params(direction='out', length=6, width=2)

<Figure size 1000x800 with 0 Axes>

In [14]:
len(normal_ic50)

635

## Download Mol Files
Using openbabel to create 3D sdf files from SMILES takes a lot of time.
- Use RDKit to generate 3D molecule structures
- Use ChEMBL web service to download molecular files.
[ChEMBL Web API](https://www.ebi.ac.uk/chembl/api/data/docs)

In [15]:
todolist = []
with open('cleaned_bioactivities.txt', 'r') as file:
    for line in file.readlines():
        ligname, ligsmi, ligaff = line.strip().split(',')
        todolist.append((ligname, ligsmi))

In [16]:
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
outdir = './compounds'
if not os.path.exists(outdir):
    os.makedirs(outdir, exist_ok=True)

for ligname, ligsmi in todolist:
    sdffile = os.path.join(outdir, f'{ligname}.sdf')
    if os.path.exists(sdffile):
        continue
    mol = Chem.MolFromSmiles(ligsmi)
    if mol is None:
        print(f"SMILES Error: {ligname}, {ligsmi}")
    else:
        try:
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol)
            AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
            Chem.MolToMolFile(mol, sdffile)
        except:
            print(f"Embedding Error: {ligname}, {ligsmi}")

In [17]:
import glob
num = len(list(glob.glob(os.path.join(outdir, '*.sdf'))))
print('RDKit handable mols:', num)

RDKit handable mols: 635


## Further Filter Compounds by Properties: Lipinski's RO5

In [18]:
import rdkit.Chem as Chem
from rdkit.Chem import Descriptors

In [19]:
def check_lipinski(mol):
    mw = Descriptors.ExactMolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbond_donor = Descriptors.NumHDonors(mol)
    hbond_acc = Descriptors.NumHAcceptors(mol)
    rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    if mw >= 400 and mw <= 600 \
        and logp <= 5 \
        and hbond_donor <= 5 \
        and hbond_acc <= 10 \
        and rotatable_bonds <= 5:
        return True
    else:
         return False

In [20]:
outdir = './compounds'
subset_entries = []
minset_entries = []

with open('cleaned_bioactivities.txt', 'r') as file:
    for line in file.readlines():
        ligname, ligsmi, ligaff = line.strip().split(',')
        ligaff = float(ligaff)
        sdffile = os.path.join(outdir, f'{ligname}.sdf')
        if not os.path.exists(sdffile):
            continue
        if ligaff <= 10 or ligaff > 10000:
            subset_entries.append(line)
            lig = Chem.MolFromSmiles(ligsmi)
            mw = Descriptors.MolWt(lig)
            if mw >= 400 and mw <= 600:
                minset_entries.append(line)

print('Number of compounds for ACE with IC50<10 or >10000:', len(subset_entries))
print('Number of compounds for ACE with IC50<10 or >10000 and desired molecular weight:', len(minset_entries))

with open('cleaned_ace_bioactivities_subset.txt', 'w') as file:
    for line in subset_entries:
        file.write(line)

with open('cleaned_ace_bioactivities_minset.txt', 'w') as file:
    for line in minset_entries:
        file.write(line)

Number of compounds for ACE with IC50<10 or >10000: 303
Number of compounds for ACE with IC50<10 or >10000 and desired molecular weight: 97
